In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import base_acquire

In [7]:
r = base_acquire.scrape_github_data()

In [8]:
r


[{'repo': 'gocodeup/codeup-setup-script',
  'language': 'Shell',
  'readme_contents': '# Codeup Setup Script\n\nSetup script for Codeup students\' laptops to install the all the tools we will\nneed for the java course. We will install the following:\n\n- [xcode](https://developer.apple.com/xcode/features/): command line tools for\n  macs\n- [brew](http://brew.sh/): package manager for macs\n- [java](https://en.wikipedia.org/wiki/Java_(programming_language))\n- [tomcat](http://tomcat.apache.org/): the java webserver\n- [maven](https://maven.apache.org/): a java dependency and build management tool\n- [mysql](https://www.mysql.com/): the database we\'ll use for the class\n- [node js](https://nodejs.org/en/): a JavaScript runtime outside the browser\n- [npm](https://www.npmjs.com/): a package manager for JavaScript\n- [intellij](https://www.jetbrains.com/idea/): a Java IDE\n\nIn addition, we will:\n\n- setup ssh keys for the student\'s laptop and guide them through the process of\n  linki